In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
from matplotlib import pyplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import os
import glob
import datetime
from timeit import default_timer as timer
from influxdb import DataFrameClient
import pandas as pd
import numpy as np
import random
import math
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.metrics import mean_squared_error
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor  
from sklearn import datasets, linear_model, metrics 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR 
import matplotlib.pyplot as plt
import tqdm
import math
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from sklearn.metrics import mean_squared_error
import plotly.plotly as py
import plotly.tools as tls
host = 'localhost'
port = '8086'
user = 'mobility'
password = 'mobility!'
#dbname = 'mydb'
dbname = 'Caltrans_Loop_Detectors_Raw_Data_D7'
protocol = 'line'
client = DataFrameClient(host, port, user, password, dbname)
client.switch_database(dbname)


Bad key "text.kerning_factor" on line 4 in
C:\Users\desh669\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
### Pulling data for 3 days from the InfluxDB (Training)
i = 767470
response = client.query("select (LOOP_1_FLOW) from raw_D7 where VDS_ID = '%d' AND time >= '2019-11-25T00:00:00Z' AND time <= '2019-11-27T23:59:30Z'" % i)

### Making a dataframe
if response:
    ytrain = pd.DataFrame.from_dict(response['raw_D7'])

### Aggregating for 10 minutes
ytrain['count'] = 1
ytrain = ytrain.resample('10T', how = sum)
ytrain['load_date'] = ytrain.index
print(ytrain.head())

### checking the distribution of values
print(ytrain['count'].value_counts())
print(type(ytrain))

### log transformation for removing the seasonality
#actual_vals = ytrain['count'].values
#actual_log = np.log10(actual_vals)

#print(actual_log)
#print(actual_log).shape

In [ ]:
### Pulling data for one day from the InfluxDB (Testing)
response = client.query("select (LOOP_1_FLOW) from raw_D7 where VDS_ID = '%d' AND time >= '2019-11-28T00:00:00Z' AND time <= '2019-11-29T23:59:30Z'" % i)

if response:
    ytest = pd.DataFrame.from_dict(response['raw_D7'])

### Aggregating for 10 minutes
ytest['count'] = 1
ytest = ytest.resample('10T', how = sum)
ytest['load_date'] = ytest.index
print(ytest.head())

### checking the distribution of values
print(ytest['count'].value_counts())
print(type(ytest))

### log transformation for removing the seasonality
#actual_vals = ytest['count'].values
#actual_log = np.log10(actual_vals)

#print(actual_log)
#print(actual_log).shape

In [ ]:
train, test = ytrain['count'].values, ytest['count'].values
train_log, test_log = np.log10(train), np.log10(test)
my_order = (1, 1, 1)
my_seasonal_order = (0, 1, 1, 7)

In [ ]:
# history = [x for x in train_log]
# predictions = list()
# predict_log=list()
# for t in range(len(test_log)):
#     model = sm.tsa.SARIMAX(history, order=my_order, seasonal_order=my_seasonal_order,
#                            enforce_stationarity=False,enforce_invertibility=False)
#     model_fit = model.fit(disp=0)
#     output = model_fit.forecast()
#     predict_log.append(output[0])
#     yhat = 10**output[0]
#     predictions.append(yhat)
#     obs = test_log[t]
#     history.append(obs)
#     #print('predicted=%f, expected=%f' % (output[0], obs))
#     #error = math.sqrt(mean_squared_error(test_log, predict_log))
#     #print('Test rmse: %.3f' % error)
# # plot
# figsize=(12, 7)
# plt.figure(figsize=figsize)
# plt.plot(test,label='Actuals')
# plt.plot(predictions, color='red',label='Predicted')
# plt.legend(loc='upper right')
# plt.ylabel('Traffic Flow (Aggregated)',fontsize = 18)
# plt.title('Seasonal ARIMA for Thursday (Training on Monday to Wednesday)',fontsize = 22)
# #plt.savefig('Seasonal_Thursday_&Mon&Tue&Wed.png')
# plt.show()

In [10]:
# #### Now in order to tune the parameters we wil try to use the auto arima model
# from pyramid.arima import auto_arima
# stepwise_model = auto_arima(train_log, start_p=1, start_q=1,
#                            max_p=3, max_q=3, m=7,
#                            start_P=0, seasonal=True,
#                            d=1, D=1, trace=True,
#                            error_action='ignore',  
#                            suppress_warnings=True, 
#                            stepwise=True)
# history = [x for x in train_log]
# predictions = list()
# predict_log=list()
# for t in range(len(test_log)):
#     #model = sm.tsa.SARIMAX(history, order=my_order, seasonal_order=my_seasonal_order,enforce_stationarity=False,enforce_invertibility=False)
#     stepwise_model.fit(history)
#     output = stepwise_model.predict(n_periods=1)
#     predict_log.append(output[0])
#     yhat = 10**output[0]
#     predictions.append(yhat)
#     obs = test_log[t]
#     history.append(obs)
#     #print('predicted=%f, expected=%f' % (output[0], obs))
# #error = math.sqrt(mean_squared_error(test_log, predict_log))
# #print('Test rmse: %.3f' % error)
# # plot
# figsize=(12, 7)
# plt.figure(figsize=figsize)
# plt.plot(test,label='Actuals')
# plt.plot(predictions, color='red',label='Predicted')
# plt.ylabel('Traffic Flow (Aggregated)',fontsize = 18)
# plt.title('AUTO ARIMA for Thursday (Training on Monday to Wednesday)',fontsize = 22)
# plt.legend(loc='upper right')
# #plt.savefig('Auto_Arima_Thursday_M&T&W.png')
# plt.show()

In [ ]:
### Now let us try to create a dataframe of actuals and predicted traffic flow values

# predicted_df = pd.DataFrame()
# predicted_df['load_date'] = ytest['load_date']
# predicted_df['actuals'] = ytest['count']
# predicted_df['predicted'] = predictions

# predicted_df.reset_index(inplace=True)
# del predicted_df['index']
# print(predicted_df.head())

# predicted_df.to_csv('ARIMA_ANOMALY.csv')

In [4]:
cd C:\Users\desh669\OneDrive - PNNL\Documents\Rahul Important

C:\Users\desh669\OneDrive - PNNL\Documents\Rahul Important


In [5]:
predicted_df = pd.read_csv('ARIMA_ANOMALY.csv')

In [6]:
def detect_classify_anomalies(df,window):
    df.replace([np.inf, -np.inf], np.NaN, inplace=True)
    df.fillna(0,inplace=True)
    df['error']=df['actuals']-df['predicted']
    df['percentage_change'] = ((df['actuals'] - df['predicted']) / df['actuals']) * 100
    df['meanval'] = df['error'].rolling(window=window).mean()
    df['deviation'] = df['error'].rolling(window=window).std()
    df['-3s'] = df['meanval'] - (2 * df['deviation'])
    df['3s'] = df['meanval'] + (2 * df['deviation'])
    df['-2s'] = df['meanval'] - (1.75 * df['deviation'])
    df['2s'] = df['meanval'] + (1.75 * df['deviation'])
    df['-1s'] = df['meanval'] - (1.5 * df['deviation'])
    df['1s'] = df['meanval'] + (1.5 * df['deviation'])
    cut_list = df[['error', '-3s', '-2s', '-1s', 'meanval', '1s', '2s', '3s']]
    cut_values = cut_list.values
    cut_sort = np.sort(cut_values)
    df['impact'] = [(lambda x: np.where(cut_sort == df['error'][x])[1][0])(x) for x in
                               range(len(df['error']))]
    severity = {0: 3, 1: 2, 2: 1, 3: 0, 4: 0, 5: 1, 6: 2, 7: 3}
    region = {0: "NEGATIVE", 1: "NEGATIVE", 2: "NEGATIVE", 3: "NEGATIVE", 4: "POSITIVE", 5: "POSITIVE", 6: "POSITIVE",
              7: "POSITIVE"}
    df['color'] =  df['impact'].map(severity)
    df['region'] = df['impact'].map(region)
    df['anomaly_points'] = np.where(df['color'] == 3, df['error'], np.nan)
    df = df.sort_values(by='load_date', ascending=False)
    df.load_date = pd.to_datetime(df['load_date'].astype(str), format="%Y-%m-%d")

    return df

In [7]:
def plot_anomaly(df,metric_name):
    #error = pd.DataFrame(Order_results.error.values)
    #df = df.sort_values(by='load_date', ascending=False)
    #df.load_date = pd.to_datetime(df['load_date'].astype(str), format="%Y%m%d")
    dates = df.load_date
    #meanval = error.rolling(window=window).mean()
    #deviation = error.rolling(window=window).std()
    #res = error

    #upper_bond=meanval + (2 * deviation)
    #lower_bond=meanval - (2 * deviation)

    #anomalies = pd.DataFrame(index=res.index, columns=res.columns)
    #anomalies[res < lower_bond] = res[res < lower_bond]
    #anomalies[res > upper_bond] = res[res > upper_bond]
    bool_array = (abs(df['anomaly_points']) > 0)



    #And a subplot of the Actual Values.
    actuals = df["actuals"][-len(bool_array):]
    anomaly_points = bool_array * actuals
    anomaly_points[anomaly_points == 0] = np.nan

    #Order_results['meanval']=meanval
    #Order_results['deviation']=deviation

    color_map= {0: "'rgba(228, 222, 249, 0.65)'", 1: "yellow", 2: "orange", 3: "red"}
    table = go.Table(
    domain=dict(x=[0, 1],
                y=[0, 0.3]),
    columnwidth=[1, 2 ],
    #columnorder=[0, 1, 2,],
    header = dict(height = 20,
                  values = [['<b>Date</b>'],['<b>Actual Values </b>'],
                            ['<b>Predicted</b>'], ['<b>% Difference</b>'],['<b>Severity (0-3)</b>']],
                 font = dict(color=['rgb(45, 45, 45)'] * 5, size=14),
                  fill = dict(color='#d562be')),
    cells = dict(values = [df.round(3)[k].tolist() for k in ['load_date', 'actuals', 'predicted',
                                                               'percentage_change','color']],
                 line = dict(color='#506784'),
                 align = ['center'] * 5,
                 font = dict(color=['rgb(40, 40, 40)'] * 5, size=12),
                 #format = [None] + [",.4f"] + [',.4f'],

                 #suffix=[None] * 4,
                 suffix=[None] + [''] + [''] + ['%'] + [''],
                 height = 27,
                 #fill = dict(color=['rgb(235, 193, 238)', 'rgba(228, 222, 249, 0.65)']))
                 fill=dict(color=  # ['rgb(245,245,245)',#unique color for the first column
                      [df['color'].map(color_map)],
                      )
    ))


    #df['ano'] = np.where(df['color']==3, df['error'], np.nan)

    anomalies = go.Scatter(name="Anomaly",
                       x=dates,
                       xaxis='x1',
                       yaxis='y1',
                       y=df['anomaly_points'],
                       mode='markers',
                       marker = dict(color ='red',
                      size = 11,line = dict(
                                         color = "red",
                                         width = 2)))

    upper_bound = go.Scatter(hoverinfo="skip",
                         x=dates,
                         showlegend =False,
                         xaxis='x1',
                         yaxis='y1',
                         y=df['3s'],
                         marker=dict(color="#444"),
                         line=dict(
                             color=('rgb(23, 96, 167)'),
                             width=2,
                             dash='dash'),
                         fillcolor='rgba(68, 68, 68, 0.3)',
                         fill='tonexty')

    lower_bound = go.Scatter(name='Confidence Interval',
                          x=dates,
                         xaxis='x1',
                         yaxis='y1',
                          y=df['-3s'],
                          marker=dict(color="#444"),
                          line=dict(
                              color=('rgb(23, 96, 167)'),
                              width=2,
                              dash='dash'),
                          fillcolor='rgba(68, 68, 68, 0.3)',
                          fill='tonexty')

    Actuals = go.Scatter(name= 'Actuals',
                     x= dates,
                     y= df['actuals'],
                    xaxis='x2', yaxis='y2',
                     mode='line',
                     marker=dict(size=12,
                                 line=dict(width=1),
                                 color="blue"))

    Predicted = go.Scatter(name= 'Predicted',
                     x= dates,
                     y= df['predicted'],
                    xaxis='x2', yaxis='y2',
                     mode='line',
                     marker=dict(size=12,
                                 line=dict(width=1),
                                 color="orange"))



    # create plot for error...
    Error = go.Scatter(name="Error",
                   x=dates, y=df['error'],
                   xaxis='x1',
                   yaxis='y1',
                   mode='line',
                   marker=dict(size=12,
                               line=dict(width=1),
                               color="red"),
                   text="Error")



    anomalies_map = go.Scatter(name = "anomaly actual",
                                   showlegend=False,
                                   x=dates,
                                   y=anomaly_points,
                                   mode='markers',
                                   xaxis='x2',
                                   yaxis='y2',
                                    marker = dict(color ="red",
                                  size = 11,
                                 line = dict(
                                     color = "red",
                                     width = 2)))

    Mvingavrg = go.Scatter(name="Moving Average",
                           x=dates,
                           y=df['meanval'],
                           mode='line',
                           xaxis='x1',
                           yaxis='y1',
                           marker=dict(size=12,
                                       line=dict(width=1),
                                       color="green"),
                           text="Moving average")




    axis=dict(
    showline=True,
    zeroline=False,
    showgrid=True,
    mirror=True,
    ticklen=4,
    gridcolor='#ffffff',
    tickfont=dict(size=10))

    layout = dict(
    width=1000,
    height=865,
    autosize=False,
    title= metric_name,
    margin = dict(t=75),
    showlegend=True,
    xaxis1=dict(axis, **dict(domain=[0, 1], anchor='y1', showticklabels=True)),
    xaxis2=dict(axis, **dict(domain=[0, 1], anchor='y2', showticklabels=True)),
    yaxis1=dict(axis, **dict(domain=[2 * 0.21 + 0.20 + 0.09, 1], anchor='x1', hoverformat='.2f')),
    yaxis2=dict(axis, **dict(domain=[0.21 + 0.12, 2 * 0.31 + 0.02], anchor='x2', hoverformat='.2f')))






    fig = go.Figure(data = [table,anomalies,anomalies_map,
                        upper_bound,lower_bound,Actuals,Predicted,
                        Mvingavrg,Error], layout = layout)

    iplot(fig)
    plt.show()

In [9]:
classify_df=detect_classify_anomalies(predicted_df,7)
classify_df.reset_index(inplace=True)
del classify_df['index']
classify_df.head()
plot_anomaly(classify_df.iloc[:-6,:],"Anomalies in Traffic Flow using Arima Models")